In [5]:
import pandas as pd

path = "../data/interim/firm_years_labeled.parquet"
df = pd.read_parquet(path)

df.shape, df.columns.tolist()

((10764, 18),
 ['cik',
  'name',
  'city',
  'state',
  'sic',
  'incorp_state',
  'filing_type',
  'fye',
  'filing_date',
  'reporting_date',
  'url',
  'mda',
  'is_fraud',
  'fraud_aaer_no',
  'fraud_start',
  'fraud_end',
  'fraud_case_datetime',
  'fraud_release_no'])

In [6]:
df.dtypes
df.head(3)

,cik,name,city,state,sic,incorp_state,filing_type,fye,filing_date,reporting_date,url,mda,is_fraud,fraud_aaer_no,fraud_start,fraud_end,fraud_case_datetime,fraud_release_no
0,896493,DIGITAL POWER CORP,FREMONT,CA,3679,CA,10-K,31-12,25-03-2010,31-12-2009,https://www.sec.gov/Archives/edgar/data/896493...,ITEM 7. \n\nMANAGEMENT&#8217;S DISCUSSION AND...,1,AAER-4440,01-2016,12-2021,2023-08-15T12:​34:​52Z,"33-11222, 34-98131"
1,876883,MDC PARTNERS INC,NEW YORK,NY,7311,A6,10-K,31-12,26-02-2016,31-12-2015,https://www.sec.gov/Archives/edgar/data/876883...,Item 7. Management&#8217;s Discussion and Ana...,1,AAER-3849,01-2009,12-2014,2017-01-18T08:24:41-05:00,"33-10283, 34-79823"
2,886035,GENERAL CABLE CORP /DE/,HIGHLAND HEIGHTS,KY,3357,DE,10-K405,31-12,30-03-1998,31-12-1997,https://www.sec.gov/Archives/edgar/data/886035...,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS ...,1,AAER-3840,01-2008,06-2012,2016-12-29T08:22:51-05:00,34-79702


In [7]:
df["is_fraud"].value_counts(dropna=False)
df["is_fraud"].value_counts(normalize=True, dropna=False)

is_fraud
1    0.987923
0    0.012077
Name: proportion, dtype: float64

In [8]:
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
miss_tbl = (
    pd.DataFrame({"missing": missing, "missing_pct": missing_pct})
      .sort_values("missing_pct", ascending=False)
)
miss_tbl.head(20)

,missing,missing_pct
mda,1448,13.45
fraud_release_no,130,1.21
fraud_case_datetime,130,1.21
fraud_end,130,1.21
fraud_start,130,1.21
fraud_aaer_no,130,1.21
name,0,0.00
is_fraud,0,0.00
url,0,0.00
cik,0,0.00


In [9]:
mda_present = df["mda"].notna()
mda_present.mean(), mda_present.sum()

(0.8654775176514307, 9316)

In [10]:
tmp = df.copy()
tmp["mda_len"] = tmp["mda"].fillna("").str.len()
tmp["mda_word_count"] = tmp["mda"].fillna("").str.split().str.len()

tmp[["mda_len", "mda_word_count"]].describe()

,mda_len,mda_word_count
count,1.076400e+04,10764.000000
mean,6.496765e+04,9520.269417
std,7.565789e+04,11016.827918
min,0.000000e+00,0.000000
25%,5.347500e+02,76.000000
50%,5.198500e+04,7645.000000
75%,9.260950e+04,13672.500000
max,1.251239e+06,188443.000000


In [11]:
tmp.groupby("is_fraud")[["mda_len", "mda_word_count"]].describe()

mda_len                                                       \
            count           mean            std  min     25%       50%   
is_fraud                                                                 
0           130.0  128314.815385  121367.886801  0.0  339.00  103927.0   
1         10634.0   64193.231804   74603.828643  0.0  538.25   51719.5   

                               mda_word_count                              \
                75%        max          count          mean           std   
is_fraud                                                                    
0         201637.75   525830.0          130.0  18606.307692  17505.927635   
1          91927.00  1251239.0        10634.0   9409.193154  10868.061070   

                                                   
          min    25%      50%       75%       max  
is_fraud                                           
0         0.0  46.00  15377.0  28936.00   77868.0  
1         0.0  76.25   7585.0  13543.25  188443.0

In [12]:
tmp["filing_type"].value_counts(dropna=False).head(20)
tmp["sic"].value_counts(dropna=False).head(20)
tmp["state"].value_counts(dropna=False).head(20)

state
NY    1400
CA    1371
TX     944
IL     608
MA     500
PA     497
FL     495
NJ     470
OH     453
VA     324
CT     320
GA     291
MI     290
NC     251
IN     190
MO     168
MD     160
WA     144
TN     122
OR     119
Name: count, dtype: int64

In [13]:
for c in ["filing_date", "reporting_date", "fraud_start", "fraud_end"]:
    tmp[c] = pd.to_datetime(tmp[c], errors="coerce")

tmp[["filing_date", "reporting_date", "fraud_start", "fraud_end"]].agg(["min", "max", "count"])

/var/folders/82/60hb6pbx3w78l4r_xzxplqzc0000gp/T/ipykernel_15365/2741886077.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  tmp[c] = pd.to_datetime(tmp[c], errors="coerce")
/var/folders/82/60hb6pbx3w78l4r_xzxplqzc0000gp/T/ipykernel_15365/2741886077.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  tmp[c] = pd.to_datetime(tmp[c], errors="coerce")
/var/folders/82/60hb6pbx3w78l4r_xzxplqzc0000gp/T/ipykernel_15365/2741886077.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tmp[c] = pd.to_datetime(tmp[c], errors="coerce")
/var/folders/82/60hb6pbx3w78l4r_xzxplqzc0000gp/T/ipykernel_15365/2741886077.py:2: UserWarning: 

,filing_date,reporting_date,fraud_start,fraud_end
min,1994-01-25 00:00:00,1992-12-31 00:00:00,1983-01-01 00:00:00,1993-12-01 00:00:00
max,2025-05-20 00:00:00,2025-03-31 00:00:00,2022-04-01 00:00:00,2022-12-01 00:00:00
count,10764,10764,10634,10634


In [14]:
from pathlib import Path
from ydata_profiling import ProfileReport

Path("reports").mkdir(exist_ok=True)

cols = ["cik","sic","filing_type","state","mda","is_fraud"]
dfp = df[cols].copy()

profile = ProfileReport(
    dfp,
    title="Fraud Detection – EDA Profile",
    vars={
        "num": {"chi_squared_threshold": 0.0},   # disable chi-square [page:1]
        "cat": {"chi_squared_threshold": 0.0},   # disable chi-square for categoricals [page:1]
    },
    correlations=None,  # disables all correlation matrices (often speeds up + avoids issues) [page:1]
)

profile.to_file("reports/eda_profile.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 6/6 [01:26<00:00, 14.39s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]